# Text Generation in Shakespeare's Style

**Import Dependencies**

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
import numpy as np

**Download Shakespeare Dataset**



In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt \
    -O sonnets.txt

--2022-01-25 04:49:00--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/sonnets.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 108.177.126.128, 108.177.127.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘sonnets.txt’

sonnets.txt         100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2022-01-25 04:49:01 (77.6 MB/s) - ‘sonnets.txt’ saved [93578/93578]



**Define Tokenizer and prepare training data**

In [ ]:
tokenizer = Tokenizer()

data= open('sonnets.txt').read()

corpus = data.lower().split('\n')

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print(total_words)

In [ ]:
input_sequences = []

for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  # print("LIST = "+str(token_list))
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    # print(n_gram_sequence)
    input_sequences.append(n_gram_sequence)

# print(len(input_sequences))

# #pad sequences
max_sequence_len = max([len(seq) for seq in input_sequences])

print(max_sequence_len, total_words)

input_sequences = np.array(pad_sequences(input_sequences, padding='pre', maxlen=max_sequence_len))

#create predictors and labels
xs, labels = input_sequences[:,:-1], input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)


11 3211


**Define the Model Architecture and start training**

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(96)))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 113, 100)          284400    
_________________________________________________________________
bidirectional (Bidirectional (None, 113, 300)          301200    
_________________________________________________________________
dropout (Dropout)            (None, 113, 300)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 192)               304896    
_________________________________________________________________
dense (Dense)                (None, 1422)              274446    
_________________________________________________________________
dense_1 (Dense)              (None, 2844)              4047012   
Total params: 5,211,954
Trainable params: 5,211,954
Non-trainable params: 0
______________________________________________

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
390/390 [==============================] - 58s 128ms/step - loss: 5.4444 - acc: 0.0805
Epoch 2/100
390/390 [==============================] - 46s 118ms/step - loss: 5.2167 - acc: 0.0906
Epoch 3/100
390/390 [==============================] - 45s 116ms/step - loss: 5.0802 - acc: 0.0984
Epoch 4/100
390/390 [==============================] - 45s 116ms/step - loss: 4.9581 - acc: 0.1105
Epoch 5/100
390/390 [==============================] - 45s 115ms/step - loss: 4.8565 - acc: 0.1217
Epoch 6/100
390/390 [==============================] - 44s 113ms/step - loss: 4.7616 - acc: 0.1273
Epoch 7/100
390/390 [==============================] - 43s 111ms/step - loss: 4.6784 - acc: 0.1340
Epoch 8/100
390/390 [==============================] - 43s 111ms/step - loss: 4.5942 - acc: 0.1418
Epoch 9/100
390/390 [==============================] - 43s 110ms/step - loss: 4.5218 - acc: 0.1497
Epoch 10/100
390/390 [==============================] - 43s 110ms/step - loss: 4.4440 - acc: 0.1577
Epoch 11/

KeyboardInterrupt: ignored

**Generate 100 next words with an initial sample sentence**

In [ ]:

def predict_next_words(seed_text, next_words):
  for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predict_x=model.predict(token_list) 
    predicted=np.argmax(predict_x,axis=1)
    # predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word

  print(seed_text)
  return seed_text

In [ ]:
seed_text = "1THREEPIO"
next_words = 100


generated_text = predict_next_words(seed_text, next_words)

1THREEPIO han red leader rendezvous at mark six one rebel base one coming in listen and you are to happen any well upset the force will be with you always sounds like they than word in here this is the jump to light speed and scheduled to fry the jedi knights were the force is done if he was even enough of the jump to light speed is under control you've ought to be able to get us here the detention planet of anchorhead this is kenobi now you're nothing if five of my father like you to dump it but


In [ ]:
seed_text = "why lovest thou that which thou receivest not gladly,"
generated_text = predict_next_words(seed_text, next_words)

why lovest thou that which thou receivest not gladly, gladly change things life dead brow new lack lie to my life still doth some cross ' bred another youth their night ' still doth date rage of life but therein quite gone straight lack dead face dead young thee hour mine eye some ill alone doth stay thy hate must care you moan still o'er hell or woe remain new fire ' must do bring all rest fitted tongue forgot thus three near contents stay your place must seen have near forth o'er truth still truly dwell to groan things tend ill doth hide to thee in me bring
